In [ ]:
import matplotlib.pyplot as plt
import numpy
import pandas

import seaborn
seaborn.set_context('talk')

from sqlalchemy import create_engine, text

**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Exploración y Curación de Datos**

*Edición 2023*

----

# Trabajo práctico entregable - parte 1

En esta notebook, vamos a cargar el conjunto de datos de [la compentencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) sobre estimación de precios de ventas de propiedades en Melbourne, Australia.

Utilizaremos el conjunto de datos reducido producido por [DanB](https://www.kaggle.com/dansbecker). Hemos subido una copia a un servidor de la Universidad Nacional de Córdoba para facilitar su acceso remoto.

In [ ]:
import plotly
plotly.__version__


'5.13.1'

In [ ]:
melb_df = pandas.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')

interesting_cols = [
  'description', 'neighborhood_overview',
  'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
  'price', 'weekly_price', 'monthly_price',
  'latitude', 'longitude']

airbnb_df = pandas.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv',
    usecols=interesting_cols)

<ipython-input-3-9288206b545b>:10: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  airbnb_df = pandas.read_csv(


In [ ]:
melb_df[:3]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0


In [ ]:
airbnb_df[:3]

,description,neighborhood_overview,street,neighborhood,city,suburb,state,zipcode,latitude,longitude,price,weekly_price,monthly_price
0,"House: Clean, New, Modern, Quite, Safe. 10Km f...",Very safe! Family oriented. Older age group.,"Bulleen, VIC, Australia",Balwyn North,Manningham,Bulleen,VIC,3105,-37.772684,145.092133,60,NaN,NaN
1,A large air conditioned room with queen spring...,This hip area is a crossroads between two grea...,"Brunswick East, VIC, Australia",Brunswick,Moreland,Brunswick East,VIC,3057,-37.766505,144.980736,35,200.0,803.0
2,RIGHT IN THE HEART OF ST KILDA! It doesn't get...,A stay at our apartment means you can enjoy so...,"St Kilda, VIC, Australia",St Kilda,Port Phillip,St Kilda,VIC,3182,-37.859755,144.977369,159,1253.0,4452.0


## Ejercicio 1 SQL: 

1. Crear una base de datos en SQLite utilizando la libreria [SQLalchemy](https://stackoverflow.com/questions/2268050/execute-sql-from-file-in-sqlalchemy).
https://docs.sqlalchemy.org/en/14/core/engines.html#sqlite

2. Ingestar los datos provistos en 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv' en una tabla y el dataset generado en clase con datos de airbnb y sus precios por codigo postal en otra.

3. Implementar consultas en SQL que respondan con la siguiente información:

    - cantidad de registros totales por ciudad.
    - cantidad de registros totales por barrio y ciudad.

4. Combinar los datasets de ambas tablas ingestadas utilizando el comando JOIN de SQL  para obtener un resultado similar a lo realizado con Pandas en clase.  



In [ ]:
# 1)

engine = create_engine('sqlite:///data.sqlite3', echo=True)

# 2)

melb_df.to_sql('melb_data', con=engine, if_exists="replace")
airbnb_df.to_sql('airbnb_data', con=engine, if_exists="replace")


3.1) Cantidad de registros totales por ciudad

In [ ]:
# melb_data no tiene datos de ciudad, así que hacemos un query para suburbios. airbnb sí tiene datos de ciudad,
# así que ahí sí agrupamos por ciudad.

query1 = """
SELECT Suburb,
       Count(*)
FROM   melb_data
GROUP  BY Suburb 
"""

with engine.connect() as con:
      rs = con.execute(text(query1))
      df_rs = pandas.DataFrame(rs.fetchall())
print(df_rs)

2023-06-08 20:04:02,350 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-06-08 20:04:02,355 INFO sqlalchemy.engine.Engine 
SELECT Suburb,
       Count(*)
FROM   melb_data
GROUP  BY Suburb 



INFO:sqlalchemy.engine.Engine:
SELECT Suburb,
       Count(*)
FROM   melb_data
GROUP  BY Suburb 



2023-06-08 20:04:02,361 INFO sqlalchemy.engine.Engine [generated in 0.01081s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.01081s] ()


2023-06-08 20:04:02,381 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


           Suburb  Count(*)
0      Abbotsford        56
1      Aberfeldie        44
2    Airport West        67
3       Albanvale         6
4     Albert Park        69
..            ...       ...
309    Wonga Park         1
310  Wyndham Vale         4
311     Yallambie        24
312    Yarra Glen         1
313    Yarraville       164

[314 rows x 2 columns]


In [ ]:
query2 = """
SELECT city,
       Count(*)
FROM   airbnb_data
GROUP  BY city
"""

with engine.connect() as con:
      rs = con.execute(text(query2))
      df_rs = pandas.DataFrame(rs.fetchall())
print(df_rs)

2023-06-08 20:11:02,922 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-06-08 20:11:02,927 INFO sqlalchemy.engine.Engine 
SELECT city,
       Count(*)
FROM   airbnb_data
GROUP  BY city



INFO:sqlalchemy.engine.Engine:
SELECT city,
       Count(*)
FROM   airbnb_data
GROUP  BY city



2023-06-08 20:11:02,933 INFO sqlalchemy.engine.Engine [generated in 0.01062s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.01062s] ()


2023-06-08 20:11:02,966 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


                 city  Count(*)
0             Banyule       203
1             Bayside       375
2          Boroondara       664
3            Brimbank       108
4            Cardinia       123
5               Casey       153
6             Darebin       698
7           Frankston       177
8           Glen Eira       631
9   Greater Dandenong       147
10        Hobsons Bay       239
11               Hume       170
12           Kingston       309
13               Knox       175
14         Manningham       313
15        Maribyrnong       436
16          Maroondah       115
17          Melbourne      7368
18             Melton        95
19             Monash       571
20      Moonee Valley       344
21           Moreland       967
22          Nillumbik        88
23       Port Phillip      2808
24        Stonnington      1621
25         Whitehorse       614
26         Whittlesea       137
27            Wyndham       426
28              Yarra      2049
29       Yarra Ranges       771


3.2) Cantidad de registros totales por barrio y ciudad

In [ ]:
# Nuevamente, para el caso de melb_data usamos variables similares a las pedidas (Suburb y CouncilArea)

query3 = """
SELECT Suburb,
       CouncilArea,
       Count(*)
FROM   melb_data
WHERE
CouncilArea != 'None'
and Suburb != 'None'
GROUP  BY Suburb,
          CouncilArea 
"""

with engine.connect() as con:
      rs = con.execute(text(query3))
      df_rs = pandas.DataFrame(rs.fetchall())
print(df_rs)

2023-06-08 20:09:38,348 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-06-08 20:09:38,354 INFO sqlalchemy.engine.Engine 
SELECT Suburb,
       CouncilArea,
       Count(*)
FROM   melb_data
WHERE
CouncilArea != 'None'
and Suburb != 'None'
GROUP  BY Suburb,
          CouncilArea 



INFO:sqlalchemy.engine.Engine:
SELECT Suburb,
       CouncilArea,
       Count(*)
FROM   melb_data
WHERE
CouncilArea != 'None'
and Suburb != 'None'
GROUP  BY Suburb,
          CouncilArea 



2023-06-08 20:09:38,357 INFO sqlalchemy.engine.Engine [generated in 0.00940s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.00940s] ()


2023-06-08 20:09:38,393 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


           Suburb    CouncilArea  Count(*)
0      Abbotsford          Yarra        55
1      Aberfeldie  Moonee Valley        40
2    Airport West  Moonee Valley        61
3       Albanvale       Brimbank         4
4     Albert Park   Port Phillip        64
..            ...            ...       ...
333    Wonga Park     Manningham         1
334  Wyndham Vale        Wyndham         3
335     Yallambie        Banyule        23
336    Yarra Glen   Yarra Ranges         1
337    Yarraville    Maribyrnong       154

[338 rows x 3 columns]


In [ ]:
# En el caso de airbnb_data, acá sí podemos usar las variables solicitadas

query4 = """
SELECT city,
       neighborhood,
       Count(*)
FROM   airbnb_data
WHERE
city != 'None'
and neighborhood != 'None'
GROUP  BY city,
          neighborhood 
"""

with engine.connect() as con:
      rs = con.execute(text(query4))
      df_rs = pandas.DataFrame(rs.fetchall())
print(df_rs)

2023-06-08 20:19:36,503 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-06-08 20:19:36,515 INFO sqlalchemy.engine.Engine 
SELECT city,
       neighborhood,
       Count(*)
FROM   airbnb_data
WHERE
city != 'None'
and neighborhood != 'None'
GROUP  BY city,
          neighborhood 



INFO:sqlalchemy.engine.Engine:
SELECT city,
       neighborhood,
       Count(*)
FROM   airbnb_data
WHERE
city != 'None'
and neighborhood != 'None'
GROUP  BY city,
          neighborhood 



2023-06-08 20:19:36,517 INFO sqlalchemy.engine.Engine [generated in 0.01409s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.01409s] ()


2023-06-08 20:19:36,690 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


        city    neighborhood  Count(*)
0    Banyule      Alphington        16
1    Banyule         Preston         8
2    Banyule       Thornbury         5
3    Bayside       Bentleigh         1
4    Bayside        Brighton       203
..       ...             ...       ...
99     Yarra  East Melbourne         8
100    Yarra         Fitzroy       381
101    Yarra   Fitzroy North       225
102    Yarra       Northcote         6
103    Yarra        Richmond       547

[104 rows x 3 columns]


4) Combinacion de los datasets

In [ ]:
query5 = """
SELECT postcode,
       zipcode,
       address,
       avg_price
FROM   melb_data
       INNER JOIN (SELECT Avg(price) AS avg_price,
                          zipcode
                   FROM   airbnb_data
                   WHERE  zipcode != ''
                   GROUP  BY zipcode)
               ON zipcode = postcode
LIMIT  10 
"""

with engine.connect() as con:
      rs = con.execute(text(query5))
      df_rs = pandas.DataFrame(rs.fetchall())
print(df_rs)

2023-06-08 20:21:41,180 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-06-08 20:21:41,184 INFO sqlalchemy.engine.Engine 
SELECT postcode,
       zipcode,
       address,
       avg_price
FROM   melb_data
       INNER JOIN (SELECT Avg(price) AS avg_price,
                          zipcode
                   FROM   airbnb_data
                   WHERE  zipcode != ''
                   GROUP  BY zipcode)
               ON zipcode = postcode
LIMIT  10 



INFO:sqlalchemy.engine.Engine:
SELECT postcode,
       zipcode,
       address,
       avg_price
FROM   melb_data
       INNER JOIN (SELECT Avg(price) AS avg_price,
                          zipcode
                   FROM   airbnb_data
                   WHERE  zipcode != ''
                   GROUP  BY zipcode)
               ON zipcode = postcode
LIMIT  10 



2023-06-08 20:21:41,191 INFO sqlalchemy.engine.Engine [generated in 0.01096s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.01096s] ()


2023-06-08 20:21:41,244 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


   Postcode zipcode                    Address   avg_price
0    3000.0    3000            1/5 Davisons Pl  154.110731
1    3000.0    3000  1001/639 Little Bourke St  154.110731
2    3000.0    3000     1003/265 Exhibition St  154.110731
3    3000.0    3000        103/565 Flinders St  154.110731
4    3000.0    3000     1101/265 Exhibition St  154.110731
5    3000.0    3000       1201/565 Flinders St  154.110731
6    3000.0    3000       1206/483 Swanston St  154.110731
7    3000.0    3000         14/2 Exhibition St  154.110731
8    3000.0    3000        1501/87 Franklin St  154.110731
9    3000.0    3000      1615/250 Elizabeth St  154.110731


## Ejercicio 2 - Pandas: 

1. Seleccionar un subconjunto de columnas que les parezcan relevantes al problema de predicción del valor de la propiedad. Justificar las columnas seleccionadas y las que no lo fueron.
 - Eliminar los valores extremos que no sean relevantes para la predicción de valores de las propiedades.

 
2. Agregar información adicional respectiva al entorno de una propiedad a partir del [conjunto de datos de AirBnB](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) utilizado en el práctico. 
  1. Seleccionar qué variables agregar y qué combinaciones aplicar a cada una. Por ejemplo, pueden utilizar solo la columna `price`, o aplicar múltiples transformaciones como la mediana o el mínimo.
  1. Utilizar la variable zipcode para unir los conjuntos de datos. Sólo incluir los zipcodes que tengan una cantidad mínima de registros (a elección) como para que la información agregada sea relevante.
  2. Investigar al menos otras 2 variables que puedan servir para combinar los datos, y justificar si serían adecuadas o no. Pueden asumir que cuentan con la ayuda de anotadores expertos para encontrar equivalencias entre barrios o direcciones, o que cuentan con algoritmos para encontrar las n ubicaciones más cercanas a una propiedad a partir de sus coordenadas geográficas. **NO** es necesario que realicen la implementación.

Pueden leer otras columnas del conjunto de AirBnB además de las que están en `interesting_cols`, si les parecen relevantes.

Solucion

1)

In [ ]:
relevant_cols_melb = ['Rooms' , 'Price', 'Bathroom', 'Car', 'Landsize','BuildingArea','YearBuilt','Regionname', 'Suburb', 'Postcode']
filtered_melb_df = melb_df[relevant_cols_melb]
#filtered_melb_df = melb_df[relevant_cols_melb].dropna()
filtered_melb_df

,Rooms,Price,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Regionname,Suburb,Postcode
0,2,1480000.0,1.0,1.0,202.0,NaN,NaN,Northern Metropolitan,Abbotsford,3067.0
1,2,1035000.0,1.0,0.0,156.0,79.0,1900.0,Northern Metropolitan,Abbotsford,3067.0
2,3,1465000.0,2.0,0.0,134.0,150.0,1900.0,Northern Metropolitan,Abbotsford,3067.0
3,3,850000.0,2.0,1.0,94.0,NaN,NaN,Northern Metropolitan,Abbotsford,3067.0
4,4,1600000.0,1.0,2.0,120.0,142.0,2014.0,Northern Metropolitan,Abbotsford,3067.0
...,...,...,...,...,...,...,...,...,...,...
13575,4,1245000.0,2.0,2.0,652.0,NaN,1981.0,South-Eastern Metropolitan,Wheelers Hill,3150.0
13576,3,1031000.0,2.0,2.0,333.0,133.0,1995.0,Western Metropolitan,Williamstown,3016.0
13577,3,1170000.0,2.0,4.0,436.0,NaN,1997.0,Western Metropolitan,Williamstown,3016.0
13578,4,2500000.0,1.0,5.0,866.0,157.0,1920.0,Western Metropolitan,Williamstown,3016.0


In [ ]:
relevant_cols_airbnb = ['price', 'weekly_price', 'monthly_price', 'zipcode']
filtered_airbnb_df = airbnb_df[relevant_cols_airbnb]

filtered_airbnb_df['zipcode'].apply(lambda zip: str(zip).replace('VIC','').strip())
filtered_airbnb_df.loc[:, 'zipcode'] = pandas.to_numeric(filtered_airbnb_df['zipcode'], errors='coerce')
filtered_airbnb_df = filtered_airbnb_df.dropna()
filtered_airbnb_df

<ipython-input-17-657111e26b2e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_airbnb_df.loc[:, 'zipcode'] = pandas.to_numeric(filtered_airbnb_df['zipcode'], errors='coerce')
<ipython-input-17-657111e26b2e>:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  filtered_airbnb_df.loc[:, 'zipcode'] = pandas.to_numeric(filtered_airbnb_df['zipcode'], errors='coerce')


,price,weekly_price,monthly_price,zipcode
1,35,200.0,803.0,3057.0
2,159,1253.0,4452.0,3182.0
3,50,250.0,920.0,3071.0
4,69,400.0,1900.0,3183.0
6,50,335.0,1400.0,3073.0
...,...,...,...,...
18924,51,400.0,1550.0,3042.0
19323,47,528.0,2000.0,3105.0
19600,100,550.0,1990.0,3016.0
19609,79,2500.0,9000.0,3006.0


In [ ]:
airbnb_price_by_zipcode = filtered_airbnb_df[relevant_cols_airbnb].groupby('zipcode')\
  .agg({'price': ['mean', 'count'], 'weekly_price': 'mean',
        'monthly_price': 'mean'})\
  .reset_index()
airbnb_price_by_zipcode.columns = [
  ' '.join(col).strip()
  for col in airbnb_price_by_zipcode.columns.values]
airbnb_price_by_zipcode = airbnb_price_by_zipcode.rename(
    columns={'price mean': 'airbnb_price_mean',
             'price count': 'airbnb_record_count',
             'weekly_price mean': 'airbnb_weekly_price_mean',
             'monthly_price mean': 'airbnb_monthly_price_mean'})  
airbnb_price_by_zipcode

,zipcode,airbnb_price_mean,airbnb_record_count,airbnb_weekly_price_mean,airbnb_monthly_price_mean
0,3000.0,142.217822,202,935.608911,3412.133663
1,3002.0,171.812500,16,1125.500000,3844.625000
2,3003.0,130.000000,7,743.857143,2690.857143
3,3004.0,192.769231,26,1088.384615,3859.153846
4,3006.0,190.870968,93,1230.311828,4312.150538
...,...,...,...,...,...
162,3911.0,275.000000,2,1340.000000,5425.000000
163,3975.0,129.000000,1,850.000000,3000.000000
164,3977.0,124.500000,2,245.000000,1025.000000
165,3978.0,300.000000,1,2202.000000,7770.000000


2)

In [ ]:
merged_sales_df = filtered_melb_df.merge(
    airbnb_price_by_zipcode, how='left',
    left_on='Postcode', right_on='zipcode'
)
merged_sales_df = merged_sales_df[merged_sales_df['airbnb_record_count'] > 5]
# merged_sales_df = merged_sales_df.dropna()
merged_sales_df = merged_sales_df.drop('airbnb_record_count', axis=1)
merged_sales_df = merged_sales_df.drop('Postcode', axis=1)
merged_sales_df

,Rooms,Price,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Regionname,Suburb,zipcode,airbnb_price_mean,airbnb_weekly_price_mean,airbnb_monthly_price_mean
0,2,1480000.0,1.0,1.0,202.0,NaN,NaN,Northern Metropolitan,Abbotsford,3067.0,90.892857,597.821429,2214.071429
1,2,1035000.0,1.0,0.0,156.0,79.0,1900.0,Northern Metropolitan,Abbotsford,3067.0,90.892857,597.821429,2214.071429
2,3,1465000.0,2.0,0.0,134.0,150.0,1900.0,Northern Metropolitan,Abbotsford,3067.0,90.892857,597.821429,2214.071429
3,3,850000.0,2.0,1.0,94.0,NaN,NaN,Northern Metropolitan,Abbotsford,3067.0,90.892857,597.821429,2214.071429
4,4,1600000.0,1.0,2.0,120.0,142.0,2014.0,Northern Metropolitan,Abbotsford,3067.0,90.892857,597.821429,2214.071429
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13562,3,1450000.0,1.0,1.0,312.0,NaN,NaN,Northern Metropolitan,Thornbury,3071.0,117.705882,693.823529,2578.588235
13563,3,1271000.0,1.0,2.0,470.0,151.0,1950.0,Northern Metropolitan,Thornbury,3071.0,117.705882,693.823529,2578.588235
13573,4,635000.0,2.0,1.0,662.0,172.0,1980.0,Western Metropolitan,Werribee,3030.0,78.933333,606.000000,2224.600000
13575,4,1245000.0,2.0,2.0,652.0,NaN,1981.0,South-Eastern Metropolitan,Wheelers Hill,3150.0,82.722222,522.055556,1901.611111


3) Podríamos usar CouncilArea y city para combinar ambos datasets tambien porque hay una buena interseccion entre los valores de ambos datasets.

In [ ]:
total_city_melb = len(melb_df['CouncilArea'].dropna().unique())
total_city_airbnb = len(airbnb_df['city'].dropna().unique())
intersection = len(numpy.intersect1d(melb_df['CouncilArea'].dropna(), airbnb_df['city'].dropna()))
print(f'Total cities on melbourn dataset = {total_city_melb}')
print(f'Total cities on airbnb dataset = {total_city_airbnb}')
print(f'Intersection = {intersection}')

Total cities on melbourn dataset = 33
Total cities on airbnb dataset = 30
Intersection = 30


Tambien se podria usar suburb por la misma razon

In [ ]:
total_city_melb = len(melb_df['Suburb'].dropna().unique())
total_city_airbnb = len(airbnb_df['suburb'].dropna().unique())
intersection = len(numpy.intersect1d(melb_df['Suburb'].dropna(), airbnb_df['suburb'].dropna()))
print(f'Total suburbs on melbourn dataset = {total_city_melb}')
print(f'Total suburbs on airbnb dataset = {total_city_airbnb}')
print(f'Intersection = {intersection}')

Total suburbs on melbourn dataset = 314
Total suburbs on airbnb dataset = 542
Intersection = 296


## Ejercicio 3:

Crear y guardar un nuevo conjunto de datos con todas las transformaciones realizadas anteriormente.

In [ ]:
merged_sales_df.to_csv('merged_sales_df.csv', index=None)
from google.colab import files
files.download('merged_sales_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Ejercicios opcionales:

1. Armar un script en python (archivo .py) [ETL](https://towardsdatascience.com/what-to-log-from-python-etl-pipelines-9e0cfe29950e) que corra los pasos de extraccion, transformacion y carga, armando una funcion para cada etapa del proceso y luego un main que corra todos los pasos requeridos.

2. Armar un DAG en Apache Airflow que corra el ETL. (https://airflow.apache.org/docs/apache-airflow/stable/tutorial.html)